# Creating figures for data from a single group

## Required data 

Each participant should have a separate folder where their spectra and MRS voxels should be stored. The folders should be arranged as follows:

```
project_folder
|   participants.tsv
|
└───data_folder
    |    
    └───sub-01
    |   |    spectrum.tsv
    |   |    spectrum_frequencies.tsv
    |   |    mrs-voxel_mni-space.nii.gz
    |   |    ...
    |
    └───sub-02
    |   |    spectrum.tsv
    |   |    spectrum_frequencies.tsv
    |   |    mrs-voxel_mni-space.nii.gz
    |   |    ...
    |
    |...
```

Participant IDs should be contained in the participants.tsv file. They should be in a column entitled "participant_id".  

Spectrum files should contain a single column with the preprocessed MRS spectra that you wish to display. The related frequencies file should contain a single column with the corresponding frequencies in PPM. 

In this example we use a MEGA-PRESS difference spectra created with the Gannet toolbox (http://www.gabamrs.com/). In the case of Gannet, these files must be written out from the data file created during the analysis. This step will differ depending on the analysis tools that you are using. 

MRS voxels should be in compressed NIFTI format. These should have been aligned to the MNI152 standard space with a 2mm resolution. 


## Make a voxel overlap image

First define the directories and files to be used and load in the participant IDs.

In [2]:
import os
import numpy as np
import pandas as pd

# Project directory
data_dir = '/home/niall/work/MRS-voxel-plot/how-to/'

# Directory to create figures in
fig_dir = os.path.join(data_dir,'figures')

# Check if this figure directory exists, and if not create it
if not os.path.isdir(fig_dir):
    os.mkdir(fig_dir)
    
# Define the column in which participant IDs are stored in the tsv file
ID_header = 'participant_id'

# Load in the participant IDs
subjects = pd.read_csv(data_dir+'participants.tsv', delimiter='\t')[ID_header]
n_subs = len(subjects)

# Define the voxel mask filename
mask_name = 'mrs-voxel_mni-space.nii.gz'

To create the overlap image we first need to establish the image dimensions and the affine matrix that defines its orientation. 

In [6]:
import nibabel as ni

# Use the first subject as an examplar for the dimensions and matrix
def get_mask_info(fpath):
    tmp = ni.load(fpath)
    aff = tmp.affine
    dims = tmp.shape
    return(aff,dims)

mask_file = os.path.join(data_dir,'example-data',subjects[0],mask_name)
mask_aff, mask_dims = get_mask_info()

Variable    Type      Data/Info
-------------------------------
ID_header   str       participant_id
data_dir    str       /home/niall/work/MRS-voxel-plot/how-to/
fig_dir     str       /home/niall/work/MRS-voxel-plot/how-to/figures
freq_file   str       spectrum_frequencies.txt
n_subs      int       24
np          module    <module 'numpy' from '/ho<...>kages/numpy/__init__.py'>
os          module    <module 'os' from '/home/<...>da3/lib/python3.7/os.py'>
pd          module    <module 'pandas' from '/h<...>ages/pandas/__init__.py'>
spec_file   str       spectrum.txt
subjects    Series    0     sub-01\n1     sub-0<...>icipant_id, dtype: object
